In [1]:
import gbs_functions as gbs
import gbs_functions_MN as old
from sympy import Matrix

In [2]:
display(gbs.moment([1,1,1]))
display(old.moment([1,1,1], displacement=False))

A[1, 2]*A[3, 4]*A[5, 6] + A[1, 2]*A[3, 5]*A[4, 6] + A[1, 2]*A[3, 6]*A[4, 5] + A[1, 3]*A[2, 4]*A[5, 6] + A[1, 3]*A[2, 5]*A[4, 6] + A[1, 3]*A[2, 6]*A[4, 5] + A[1, 4]*A[2, 3]*A[5, 6] + A[1, 4]*A[2, 5]*A[3, 6] + A[1, 4]*A[2, 6]*A[3, 5] + A[1, 5]*A[2, 3]*A[4, 6] + A[1, 5]*A[2, 4]*A[3, 6] + A[1, 5]*A[2, 6]*A[3, 4] + A[1, 6]*A[2, 3]*A[4, 5] + A[1, 6]*A[2, 4]*A[3, 5] + A[1, 6]*A[2, 5]*A[3, 4]

conjugate(M[1, 2])*conjugate(N[1, 3])*M[2, 3] + conjugate(M[1, 2])*conjugate(N[2, 3])*M[1, 3] + conjugate(M[1, 2])*M[1, 2]*N[3, 3] + conjugate(M[1, 3])*conjugate(N[1, 2])*M[2, 3] + conjugate(M[1, 3])*M[1, 2]*N[2, 3] + conjugate(M[1, 3])*M[1, 3]*N[2, 2] + conjugate(M[2, 3])*M[1, 2]*N[1, 3] + conjugate(M[2, 3])*M[1, 3]*N[1, 2] + conjugate(M[2, 3])*M[2, 3]*N[1, 1] + conjugate(N[1, 2])*conjugate(N[2, 3])*N[1, 3] + conjugate(N[1, 2])*N[1, 2]*N[3, 3] + conjugate(N[1, 3])*N[1, 2]*N[2, 3] + conjugate(N[1, 3])*N[1, 3]*N[2, 2] + conjugate(N[2, 3])*N[1, 1]*N[2, 3] + N[1, 1]*N[2, 2]*N[3, 3]

In [3]:
display(gbs.cumulant([1,1,1]))
display(old.cumulant([1,1,1], displacement=False))

A[1, 2]*A[3, 4]*A[5, 6] + A[1, 2]*A[3, 5]*A[4, 6] + A[1, 3]*A[2, 4]*A[5, 6] + A[1, 3]*A[2, 6]*A[4, 5] + A[1, 5]*A[2, 3]*A[4, 6] + A[1, 5]*A[2, 6]*A[3, 4] + A[1, 6]*A[2, 3]*A[4, 5] + A[1, 6]*A[2, 4]*A[3, 5]

conjugate(M[1, 2])*conjugate(N[1, 3])*M[2, 3] + conjugate(M[1, 2])*conjugate(N[2, 3])*M[1, 3] + conjugate(M[1, 3])*conjugate(N[1, 2])*M[2, 3] + conjugate(M[1, 3])*M[1, 2]*N[2, 3] + conjugate(M[2, 3])*M[1, 2]*N[1, 3] + conjugate(M[2, 3])*M[1, 3]*N[1, 2] + conjugate(N[1, 2])*conjugate(N[2, 3])*N[1, 3] + conjugate(N[1, 3])*N[1, 2]*N[2, 3]

### A new Montrealer
The following is the building of the new Montrealer function which is more general and is not restricted to squeezed states but is the covariance for a general Matrix A as long as there is no displacement and no repetition.

In [20]:
from gbs_functions import symmetric_A, block_A, symmetric_M
from gbs_functions import montrealer as montrealer_old
import numpy as np
from sympy import Matrix, expand
from itertools import product, permutations
from time import time
from scipy.special import factorial2

In [3]:
def bitstrings(n):
    for binary in map(''.join, product('01', repeat=n-1)):
        yield '0'+binary

#DOESNT WORK FOR N=1
#MAKE SURE A IS SQUARE
#MAKE SURE POWER IS WITHIN SIZE OF A
def montrealer(A, power):
    """
    Returns the cumulant of matrix A with selected powers.

    Arg:
        A (numpy.ndarray) : A covariance matrix.
        power (list) : List of indices of powers to consider (ex [1,4,5])

    Returns:
        (sympy.core.mul.Mul) : The cumulant.
    """
    equation = 0

    #reshaping matrix A
    m = int(np.shape(A)[0]/2)
    power = power + [m+i for i in power]
    power = [i-1 for i in power]
    A = A[power][:,power]
    
    #making the original 2 rows matrix
    m = int(np.shape(A)[0]/2)
    original = np.arange(1,2*m+1).reshape(2,m)

    #initial graphs
    graph1 = list(range(1,m))+[m+1]
    graph2 = list(range(m+2,2*m+1))+[m]

    #loop over all bistrings and all permutations
    for bit in bitstrings(m):
        for perm in permutations(range(1,m)):
            B =  np.copy(original)

            for i,j in enumerate(bit):
                if int(j):
                    buffer = B[0,i]
                    B[0,i] = B[1,i]
                    B[1,i] = buffer

            B = B[:,[0]+list(perm)] #first column stays in place always

            #dictionary mapping
            dico = {j:i+1 for i,j in enumerate(B.reshape(1,2*m)[0])}
            new_mapping = {dico[i]:dico[j] for i,j in zip(graph1,graph2)}

            term = 1
            for i,j in new_mapping.items():
                term *= A[i-1,j-1]

            equation += term

    return equation

### Testing

In [4]:
term = lambda m : factorial2(2*m-2, exact=True)

for m in range(2,7):
    power = list(np.arange(1,m+1))
    print(f'order {m}')
    A = symmetric_A(m)
    toc = time()
    B = montrealer(A, power)
    tic = time()
    print(f'time of single execution : {round(tic-toc,3)} s')  
    print(f'number of terms : {len(B.args)}')
    print(f'(2N-2)!! = {term(m)}')
    print(' ')

order 2
time of single execution : 0.076 s
number of terms : 2
(2N-2)!! = 2
 
order 3
time of single execution : 0.01 s
number of terms : 8
(2N-2)!! = 8
 
order 4
time of single execution : 0.107 s
number of terms : 48
(2N-2)!! = 48
 
order 5
time of single execution : 2.095 s
number of terms : 384
(2N-2)!! = 384
 
order 6
time of single execution : 140.315 s
number of terms : 3840
(2N-2)!! = 3840
 


### Reconciliation with the Montrealer

In [23]:
#display(montrealer(block_A(4, 'diagonal'), [1,2,3,4]))

mtl_old2 = expand(montrealer_old(symmetric_M(2)))
mtl_old4 = expand(montrealer_old(symmetric_M(4)))
mtl_old6 = expand(montrealer_old(symmetric_M(6)))

mtl2 = montrealer(block_A(2, 'diagonal'), [1,2])
mtl4 = montrealer(block_A(4, 'diagonal'), [1,2,3,4])
mtl6 = montrealer(block_A(6, 'diagonal'), [1,2,3,4,5,6])

print(mtl2.equals(mtl_old2))
print(mtl4.equals(mtl_old4))
print(mtl6.equals(mtl_old6))

True
True
True


In [24]:
mtl8 = montrealer(block_A(8, 'diagonal'), [1,2,3,4,5,6,7,8])
display(mtl8)

conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 5]*M[4, 7]*M[6, 8] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 5]*M[4, 8]*M[6, 7] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 6]*M[4, 7]*M[5, 8] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 6]*M[4, 8]*M[5, 7] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 7]*M[4, 5]*M[6, 8] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 7]*M[4, 6]*M[5, 8] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 8]*M[4, 5]*M[6, 7] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 3]*M[2, 8]*M[4, 6]*M[5, 7] + conjugate(M[1, 2])*conjugate(M[3, 4])*conjugate(M[5, 6])*conjugate(M[7, 8])*M[1, 4]*M[2, 5]*M[3, 7]*M[6, 8] + conjugate(